In [3]:
import os
import warnings
import cv2
import keras
import matplotlib.pyplot as plt
import matplotlib.style as style
import numpy as np
import pandas as pd
from PIL import Image
from keras import models, layers, optimizers
from keras.applications import VGG16
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import Dense, Dropout, Flatten
from keras.models import Model
from keras.preprocessing import image as image_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split

% matplotlib inline
style.use('seaborn-whitegrid')
warnings.filterwarnings(action='once')

UsageError: Line magic function `%` not found.


In [4]:
# change this with naming schematic
gestures = {'wave_':'Palm'}

gestures_map = {'Fist' : 0,
                'L': 1,
                'Okay': 2,
                'Palm': 3,
                'Peace': 4,
                }

In [5]:
def process_image(path):
    img = Image.open(path)
    img = img.resize((224, 224))
    img = np.array(img)
    return img

def process_data(X_data, y_data):
    X_data = np.array(X_data, dtype = 'float32')
    if rgb:
        pass
    else:
        X_data = np.stack((X_data,)*3, axis=-1)
    X_data /= 255
    y_data = np.array(y_data)
    y_data = to_categorical(y_data, num_classes=5)
    return X_data, y_data

def walk_file_tree(relative_path):
    X_data = []
    y_data = [] 
    for directory, subdirectories, files in os.walk(relative_path):
        for file in files:
            if not file.startswith('.') and (not file.startswith('C_')):
                path = os.path.join(directory, file)
                # need to change for naming schematic change...
                gesture_name = gestures[file[0:5]]
                y_data.append(gestures_map[gesture_name])
                X_data.append(process_image(path))   

            else:
                continue

    X_data, y_data = process_data(X_data, y_data)
    return X_data, y_data

class Data(object):
    def __init__(self):
        self.X_data = []
        self.y_data = []

    def get_data(self):
        return self.X_data, self.y_data

In [6]:
relative_path = './data_creation/data/'
rgb = True
X_data, y_data = walk_file_tree(relative_path)

In [7]:
silhouette = Data()
silhouette.X_data, silhouette.y_data = walk_file_tree(relative_path)

In [8]:
print(f'X_data shape: {X_data.shape}')
print(f'y_data shape: {y_data.shape}')

X_data shape: (55, 224, 224, 3)
y_data shape: (55, 5)


In [9]:
# X_train_rgb, X_test_rgb, y_train_rgb, y_test_rgb = train_test_split(image_rgb, y_data, test_size = 0.2, random_state=12, stratify=y_data)
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size = 0.2, random_state=12, stratify=y_data)

In [10]:
file_path = './models/temp_model.h5'
model_checkpoint = ModelCheckpoint(filepath=file_path, save_best_only=True)

early_stopping = EarlyStopping(monitor='val_accuracy',
                               min_delta=0,
                               patience=10,
                               verbose=1,
                               mode='auto',
                               restore_best_weights=True)

In [11]:
imageSize = 224

In [12]:
vgg_base = VGG16(weights='imagenet', include_top=False, input_shape=(imageSize, imageSize, 3))
optimizer1 = optimizers.Adam()

base_model = vgg_base  # Topless
# Add top layer
x = base_model.output
x = Flatten()(x)
x = Dense(128, activation='relu', name='fc1')(x)
x = Dense(128, activation='relu', name='fc2')(x)
x = Dense(128, activation='relu', name='fc3')(x)
x = Dropout(0.5)(x)
x = Dense(64, activation='relu', name='fc4')(x)
predictions = Dense(5, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# Train top layers only
for layer in base_model.layers:
    layer.trainable = False

callbacks_list = [keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=3, verbose=1)]

model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=3, batch_size=64, validation_data=(X_train, y_train), verbose=1,
          callbacks=[early_stopping, model_checkpoint])

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Train on 44 samples, validate on 44 samples
Epoch 1/3
44/44 [==============================] - 22s 495ms/step - loss: 1.3272 - accuracy: 0.4091 - val_loss: 0.0245 - val_accuracy: 1.0000
Epoch 2/3
44/44 [==============================] - 22s 508ms/step - loss: 0.1493 - accuracy: 0.9773 - val_loss: 2.1923e-05 - val_accuracy: 1.0000
Epoch 3/3
44/44 [==============================] - 19s 434ms/step - loss: 0.0035 - accuracy: 1.0000 - val_loss: 8.1279e-09 - val_accuracy: 1.0000


In [13]:
# load VGG16
# Get back the convolutional part of a VGG network trained on ImageNet

imageSize = 224
model1 = VGG16(weights='imagenet', include_top=False, input_shape=(imageSize, imageSize, 3))
optimizer1 = optimizers.Adam()

base_model = model1  # Topless
# Add top layer
x = base_model.output
x = Flatten()(x)
x = Dense(128, activation='relu', name='fc1')(x)
x = Dense(128, activation='relu', name='fc2')(x)
x = Dense(128, activation='relu', name='fc3')(x)
x = Dropout(0.5)(x)
x = Dense(64, activation='relu', name='fc4')(x)

predictions = Dense(5, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

# Train top layer
for layer in base_model.layers:
    layer.trainable = False

model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.Adam(),
              metrics=['accuracy'])

callbacks_list = [keras.callbacks.EarlyStopping(monitor='val_acc', patience=3, verbose=1)]

model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=3, batch_size=64, validation_data=(X_train, y_train), verbose=1,
          callbacks=[early_stopping, model_checkpoint])

Train on 44 samples, validate on 44 samples
Epoch 1/3
44/44 [==============================] - 21s 483ms/step - loss: 1.4677 - accuracy: 0.1818 - val_loss: 0.0592 - val_accuracy: 1.0000
Epoch 2/3
44/44 [==============================] - 19s 440ms/step - loss: 0.1640 - accuracy: 0.9773 - val_loss: 3.6035e-04 - val_accuracy: 1.0000
Epoch 3/3
44/44 [==============================] - 21s 479ms/step - loss: 0.0274 - accuracy: 1.0000 - val_loss: 1.9507e-06 - val_accuracy: 1.0000


In [15]:
model.save('models/new_temp.h5')

from keras.models import load_model
model = load_model('/Users/jipak/gesture_recognition/models/new_temp.h5')

In [16]:
def get_classification_metrics(X_test, y_test):
    pred = model.predict(X_test)
    pred = np.argmax(pred, axis=1)
    y_true = np.argmax(y_test, axis=1)
    print(confusion_matrix(y_true, pred))
    print('\n')
    print(classification_report(y_true, pred))
get_classification_metrics(X_data, y_data)

[[55]]


              precision    recall  f1-score   support

           3       1.00      1.00      1.00        55

    accuracy                           1.00        55
   macro avg       1.00      1.00      1.00        55
weighted avg       1.00      1.00      1.00        55

